In [31]:
using DelimitedFiles
using Plots
using LaTeXStrings
using Statistics

const autocorr_steps = 100;

## Want to add these params somehow

params = {"text.usetex": True,
            "font.family": "serif",
            "legend.fontsize": 10,
            "axes.labelsize": 10,
            "xtick.labelsize":10,
            "ytick.labelsize":10,
            "lines.linewidth":1,
            "patch.edgecolor": "black"
         }

plt.rcParams.update(params)
plt.style.use("seaborn-deep")

In [32]:
# Compute autocorrelation times from data

function autocorr_array(gamma, stoptime, T, seed, size)
    global autocorr_steps
    
    Mag_array = readdlm("stewart_data/L$(size)/gamma$(gamma)_stoptime$(stoptime)_T$(T)_seed$(seed)_mag.dat")    
    
    M_t_avg = 0
    M_t_var = 0
    M_t_M_ti_avg = zeros(autocorr_steps)
    M_ti_avg = zeros(autocorr_steps)
    M_ti_var = zeros(autocorr_steps)

    # Need to define these to calculate running variance
    # See https://www.johndcook.com/blog/standard_deviation/
    # for algo for calculating running variance

    M_t_var_aux = 0
    M_ti_var_aux = zeros(autocorr_steps)

    autocorr_steps = 100
    data_samples = 5000

    for t = 1:data_samples - autocorr_steps + 1

        M_t_avg += Mag_array[t]

        temp_M_t_var_aux = M_t_var_aux
        M_t_var_aux = M_t_var_aux + (Mag_array[t] - M_t_var_aux) / t

        if t != 1
            M_t_var = M_t_var + (Mag_array[t] - temp_M_t_var_aux) * (Mag_array[t] - M_t_var_aux)
            end # if

        for i = 1:autocorr_steps

            M_t_M_ti_avg[i] += Mag_array[t] * Mag_array[t+i-1]
            M_ti_avg[i] += Mag_array[t+i-1]

            temp_M_ti_var_aux = M_ti_var_aux[i]
            M_ti_var_aux[i] = M_ti_var_aux[i] + (Mag_array[t+i-1] - M_ti_var_aux[i]) / t

            if t != 1
                M_ti_var[i] = M_ti_var[i] + (
                    (Mag_array[t+i-1] - temp_M_ti_var_aux) * (Mag_array[t+i-1] - M_ti_var_aux[i]))
                end # if

            end # i
        end # t

    # Divide sum by total to get averages
    M_t_avg /= data_samples - autocorr_steps + 1
    M_ti_avg /= data_samples - autocorr_steps + 1
    M_t_M_ti_avg /= data_samples - autocorr_steps + 1

    # Need to do the same for variance
    M_t_var /= data_samples - autocorr_steps + 1
    M_ti_var /= data_samples - autocorr_steps + 1

    # Get stdevs with sqrt
    M_t_stdev = sqrt(M_t_var)
    M_ti_stdev = sqrt.(M_ti_var)
    
    return (M_t_M_ti_avg .- M_t_avg * M_ti_avg) ./ (M_t_stdev * M_ti_stdev)
end

autocorr_array (generic function with 1 method)

In [33]:
# Make plot of autocorrelation times across different system sizes

function make_autocorr_plot(gamma, stoptime, T, seed, system_sizes)
    time_array = 0:autocorr_steps - 1
    autocorr_arrays = zeros(length(system_sizes), autocorr_steps)

    for i in 1:length(system_sizes)
        autocorr = autocorr_array(gamma, stoptime, T, seed, system_sizes[i])
        autocorr_arrays[i, :] = autocorr
    end

    plot(time_array, autocorr_arrays[1, :], label = "L = 63")
    plot!(time_array, autocorr_arrays[2, :], label = "L = 64")
    plot!(time_array, autocorr_arrays[3, :], label = "L = 65")

    savefig("stewart_data/plots/mag_autocorr_seed$(seed).png")
    
end

make_autocorr_plot (generic function with 1 method)

In [34]:
system_sizes = [63, 64, 65]

for seed in [1234, 1111, 2222]
    make_autocorr_plot(0.01, 200, 0.135335, seed, system_sizes)
end

In [35]:
# Make subplots of magnetization autocorrelation

function make_autocorr_subplots(gamma, stoptime, T, seeds, system_sizes)
    time_array = 0:autocorr_steps - 1
    autocorr_arrays = zeros(length(system_sizes) * length(seeds), autocorr_steps)

    len_sizes = length(system_sizes)
    len_seeds = length(seeds)
    
    for i in 1:len_sizes
        for j in 1:len_seeds
            autocorr = autocorr_array(gamma, stoptime, T, seeds[j], system_sizes[i])
            autocorr_arrays[len_sizes * (j - 1) + i, :] = autocorr
        end
    end
    
    time_2d_array = repeat(time_array, 1, len_seeds)
    
    plot(time_2d_array, autocorr_arrays[1:len_sizes:len_sizes * len_seeds, :]',
        label = "L = 63", layout=(len_seeds, 1))
    plot!(time_2d_array, autocorr_arrays[2:len_sizes:len_sizes * len_seeds, :]',
        label = "L = 64", layout=(len_seeds, 1))
    plot!(time_2d_array, autocorr_arrays[3:len_sizes:len_sizes * len_seeds, :]',
        label = "L = 65", layout=(len_seeds, 1))

    savefig("stewart_data/plots/mag_autocorr_all_seeds.png")
    
end

make_autocorr_subplots (generic function with 1 method)

In [36]:
system_sizes = [63, 64, 65]
seeds = [1234, 1111, 2222]

make_autocorr_subplots(0.01, 200, 0.135335, seeds, system_sizes)

In [37]:
# Make plot of MC energy vs. temperature across different system sizes

function make_energy_plot(gamma, stoptime, seed, system_sizes)
    energy_arrays = zeros(length(system_sizes), stoptime + 1)

    for i in 1:length(system_sizes)
        size = system_sizes[i]
        energy = readdlm("stewart_data/L$(size)/gamma$(gamma)_stoptime$(stoptime)_seed$(seed)_energy.dat")
        energy_arrays[i, :] = energy
    end

    temp_array = readdlm("stewart_data/L64/gamma$(gamma)_stoptime$(stoptime)_temperature.dat")

    plot(temp_array', energy_arrays[1, :], label = "L = 63")
    plot!(temp_array', energy_arrays[2, :], label = "L = 64")
    plot!(temp_array', energy_arrays[3, :], label = "L = 65")

    savefig("stewart_data/plots/energy_vs_temp_seed$(seed).png")
end

make_energy_plot (generic function with 1 method)

In [38]:
system_sizes = [63, 64, 65]

for seed in [1234, 1111, 2222]
    make_energy_plot(0.01, 200, 1234, system_sizes)
end

In [39]:
# Make plot of variance in mag autocorrelations

function make_mag_var_plots(gamma, stoptime, T, seeds, system_sizes)
    time_array = 0:autocorr_steps - 1
    autocorr_arrays = zeros(length(system_sizes) * length(seeds), autocorr_steps)
    variance_arrays = zeros(length(system_sizes), autocorr_steps)

    for i in 1:length(system_sizes)
        for j in 1:length(seeds)
            autocorr = autocorr_array(gamma, stoptime, T, seeds[j], system_sizes[i])
            autocorr_arrays[length(system_sizes) * (j - 1) + i, :] = autocorr
        end
        same_size_autocorrs = autocorr_arrays[i:length(system_sizes):(length(system_sizes) * length(seeds)), :]
        variance_arrays[i, :] = var(same_size_autocorrs, dims=1)
    end

    plot(time_array, variance_arrays[1, :], label = "L = 63")
    plot!(time_array, variance_arrays[2, :], label = "L = 64")
    plot!(time_array, variance_arrays[3, :], label = "L = 65")

    savefig("stewart_data/plots/mag_autocorr_variance.png")
    
end

make_mag_var_plots (generic function with 1 method)

In [40]:
seeds = 1:99
system_sizes = [63, 64, 65]

make_mag_var_plots(0.01, 200, 0.135335, seeds, system_sizes)

ArgumentError: ArgumentError: Cannot open 'stewart_data/L63/gamma0.01_stoptime200_T0.135335_seed99_mag.dat': not a file